In [18]:
import pandas.io.data as web
import pandas as pd
import datetime
import Markowitz
import numpy as np
import math

ImportError: No module named 'Markowitz'

In [2]:
class Position:
    def __init__(self,symbol,returns,direction,in_price,weight):
        #format for dates: "YYYY-MM-dd"
        self.symbol = symbol
        self.returns = returns
        self.direction = direction
        self.in_price = in_price
        self.weight = weight
        if direction == "long":
            self.returns *= 100
        else:
            self.returns *= -100

In [35]:
class Portfolio:
    def __init__(self,assets,start_date,all_dates):
        self.start_date = start_date
        self.positions = assets
        self.symbols = [asset.symbol for asset in self.positions]
        self.returns_grid = np.array([asset.returns for asset in self.positions])
        self.weights = np.array([asset.weight for asset in self.positions])
        #historical
        self.historical_dates = all_dates
        self.historical_returns = np.dot(self.weights.transpose(),self.returns_grid)
        self.covariance_matrix = np.cov(self.returns_grid)
        self.net_variance = np.dot(self.weights.T,np.dot(self.covariance_matrix,self.weights))
        self.net_expectation = np.mean(self.historical_returns)
        #actual
        self.daily_values = calculate_profits(historical_returns,start_date,all_dates)
    
    def calculate_profits(assets,historical_returns,start_date,all_dates):
        #calculate first return taking into account price at which PM entered position
        first_return = 0
        for asset in assets:
            first_price = web.DataReader(asset.symbol,'yahoo',start_date)['Close'][0]
            r = 100*(first_price - asset.in_price)/asset.in_price
            if asset.direction != 'long':
                r = -r
            first_return = first_return + r
        #find index of first date
        for i in range(len(all_dates)-1,0,-1):
            if datetime.date(start_date) == datetime.date(all_dates[i]):
                return np.insert(historical_returns[i:len(returns_grid)],0,first_return)
                
    def optimal_portfolio(self):
        #looking back one year only. subject to change
        start_index = len(self.returns)-253
        end_index = len(self.returns)-1
        return_grid = self.returns_grid[:,start_index:end_index]
        weights, poly = Markowitz.optimal_portfolio_quad(return_grid,frontier=True)
        return {'weights':weights,'curve':poly}
    
    def compile_portfolio(self):
        in_prices = []
        directions = []
        for i in range(len(self.positions)):
            in_prices.append(self.positions[i].in_price)
            directions.append(self.positions[i].direction)
        compilation = Portfolio_Compiled(self.symbols,self.weights,self.returns,self.net_variance,self.net_expectation,self.historical_dates)
        return compilation

class Portfolio_Compiled:
    def __init__(self,tickers,start_date,initial_weights,in_prices,directions,returns,net_variance,net_expectation,all_dates):
        self.tickers = tickers
        self.start_date = start_date
        self.in_prices = in_prices
        self.directions = directions
        self.initial_weights = initial_weights
        self.net_returns = returns
        self.net_variance = net_variance
        self.net_expectation = net_expectation
        self.historical_dates = all_dates
    
    def uncompile(self,df):
        positions = []
        for i in range(self.tickers):
            positions.append(Position(self.tickers[i],df[self.tickers[i]],self.directions[i],self.in_prices[i],self.initial_weights[i]))
        portfolio = Portfolio(positions,self.start_date,self.historical_dates)
        return portfolio      

### Example : use of the data structure and efficiency frontier

In [37]:
import matplotlib.pyplot as plt
%matplotlib inline
tickers = ['YHOO','STK','GOOGL','AMZN','EBAY']
weights = [0.2,0.1,0.4,0.15,0.15]
#instantiate positions
positions = []
start_date = datetime.datetime(2010, 1, 1)
for i in range(len(tickers)):
    positions.append(Position(tickers[i],"long",30,start_date,weights[i],"NASDAQ","stock"))
#instantiate portfolio
portfolio = Portfolio(positions)

struct = portfolio.optimal_portfolio()
x1 = struct['curve'][2]
x2 = struct['curve'][1]
x3 = struct['curve'][0]
returns = np.linspace(-5,5,1000)
risks = [x1+x2*returns[i]+x3*returns[i]*returns[i] for i in range(len(returns))]

plt.plot(risks,returns)